# Laporan Machine Learning Praktikum: Data Preprocessing
**Nama:** Surya Dwi Satria  
**Kelas:** C7  
**NIM:** 434231048  

Notebook ini mengikuti langkah-langkah pada dokumen PDF Anda (import pustaka, load data, cek info, missing value, imputasi, deteksi & penanganan outlier, visualisasi, simpan hasil, serta validasi muat balik).

## 1) Import pustaka, load data, dan set tampilan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Opsi tampilan agar tabel tidak terpotong
# pd.set_option('display.max_columns', 100)

DATASET_PATH = Path('dataset_praktikum.csv')
df = pd.read_csv(DATASET_PATH)
print('Ukuran data:', df.shape)
df

## 2) Visualisasi distribusi & outlier (histogram)

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Histogram untuk data bersih
for c in num_cols:
    plt.figure()
    plt.hist(df[c].dropna(), bins=30)
    plt.title(f'Histogram {c} (sebelum cleaning)')
    plt.xlabel(c)
    plt.ylabel('Frekuensi')
    plt.show()


## 3) Cek missing value per kolom

In [ ]:
na_count = df.isna().sum().sort_values(ascending=False) # Misalkan ada 220 data, 10 data kosong, maka 10, isna() adalah fungsi untuk menghitung data kosong
na_pct = (df.isna().mean()*100).sort_values(ascending=False) # Misalkan ada 220 data, 10 data kosong, maka 10/220*100 = 4.55%
print(f"Jumlah NA per kolom: \n{na_count}")
print(f"\nPersentase NA per kolom:\n{na_pct}")

## 4) Imputasi missing value (numerik: median | kategori: mode)

In [ ]:
df_imputed = df.copy()

# df_imputed
num_cols = df_imputed.select_dtypes(include=[np.number]).columns.tolist() # Pilih kolom numerikal saja dan simpan dalam list
cat_cols = df_imputed.select_dtypes(exclude=[np.number]).columns.tolist() # Pilih kolom kategorikal saja dan simpan dalam list

# print('Kolom numerik:', num_cols)
# print('Kolom kategorikal:', cat_cols)


# # Imputasi numerik dengan median
for c in num_cols:
#     print(f'Imputasi kolom {c}')
    med = df_imputed[c].median() # Hitung median  di kolom tersebut 
    # print(f'Median kolom {c}: {med}')
    df_imputed[c] = df_imputed[c].fillna(med) # Isi NA dengan median
# fillna() adalah fungsi di pandas untuk mengisi nilai yang kosong (NaN) dalam sebuah kolom atau DataFrame dengan nilai tertentu.



# # Imputasi kategorikal dengan modus (mode)
for c in cat_cols:
    mode_val = df_imputed[c].mode(dropna=True) 
    # Mencari modus, atau nilai yang paling sering muncul di data 
    # dropna=True untuk mengabaikan nilai NaN saat mencari modus
    # print(f'Modus kolom {c}: {mode_val.iloc[0] if not mode_val.empty else "Tidak ada modus"}')
    if not mode_val.empty: # Cek apakah modus ditemukan
        df_imputed[c] = df_imputed[c].fillna(mode_val.iloc[0]) # Isi NA dengan modus iloc[0] adalah cara untuk mengakses nilai pertama dari hasil modus
print('Sisa NA setelah imputasi:')
print(df_imputed.isna().sum())

## 5) Deteksi outlier (IQR) dan Penanganan outlier (capping/clip ke batas IQR)

In [ ]:
def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    return lower, upper

# q1 adalah kuartil pertama yaitu nilai di bawah 25% data yang ada 
# q3 adalah kuartil ketiga yaitu nilai di bawah 75% data yang ada
# iqr adalah interquartile range, yaitu selisih antara kuartil ketiga dan kuartil pertama (q3 - q1)
# lower adalah batas bawah untuk mendeteksi outlier
# upper adalah batas atas untuk mendeteksi outlier

df_clean = df_imputed.copy() # Salin data hasil imputasi untuk dibersihkan dari outlier
outlier_report = {}
outlier_index = {}
for c in num_cols:
    lo, up = iqr_bounds(df_imputed[c])
    lo = max(lo, 0)  # Memastikan batas bawah tidak negatif, karena kolom umur tidak boleh negatif
    # print(f'Kolom {c}: lower={lo}, upper={up}') # debugging untuk melihat batas bawah dan atas
    mask = (df_clean[c] < lo) | (df_clean[c] > up) # Mask untuk mendeteksi outlier, | adalah operator "atau"
    outlier_report[c] = int(mask.sum()) # Untuk menyimpan jumlah data yang outlier
    outlier_index[c] = df_clean[mask].index.tolist() # Untuk menyimpan index data yang outlier
    df_clean[c] = df_clean[c].clip(lower=lo, upper=up) # Capping outlier ke batas bawah dan atas
    # clip() adalah fungsi di pandas untuk membatasi nilai dalam sebuah kolom atau DataFrame



print('Jumlah outlier per fitur (IQR):' )
print(outlier_report)

print("\nIndex data yang outlier per fitur:")
print(outlier_index)

df_clean.describe().T

## 6) Visualisasi distribusi & outlier (histogram dan boxplot)

In [ ]:

num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

# Histogram untuk data bersih
for c in num_cols:
    plt.figure()
    plt.hist(df_clean[c].dropna(), bins=30)
    plt.title(f'Histogram {c} (setelah cleaning)')
    plt.xlabel(c)
    plt.ylabel('Frekuensi')
    plt.show()

# Boxplot untuk data bersih
for c in num_cols:
    plt.figure()
    plt.boxplot(df_clean[c].dropna(), vert=True)
    plt.title(f'Boxplot {c} (setelah cleaning)')
    plt.ylabel(c)
    plt.show()

## 7) Susun dataset final (opsional: drop duplikat)

In [ ]:
# Untuk menyusun dataset final, kita bisa drop duplikat jika ada(opsional saja, tapi disarankan)
before = df_clean.shape[0] # Simpan jumlah baris sebelum drop duplikat
# print(f'Baris sebelum drop duplikat: {before}')
df_final = df_clean.drop_duplicates().reset_index(drop=True) # Drop duplikat dan reset index
after = df_final.shape[0]
print(f'Baris sebelum/sesudah drop duplikat: {before}/{after}')
df_final.head()



## 8) Summary & simpan hasil

In [ ]:
summary = df_final.describe(include='all').T
summary_path = Path('summary_stats.csv')
clean_path = Path('dataset_praktikum_clean.csv')
df_final.to_csv(clean_path, index=False) # Simpan data bersih ke file CSV tanpa menyimpan index
summary.to_csv(summary_path) # Simpan summary statistik ke file CSV
print('Tersimpan:', clean_path, 'dan', summary_path)
summary.head(10)